<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/Swallo7B_instract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers sentencepiece accelerate bitsandbytes

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained(
    "tokyotech-llm/Swallow-7b-instruct-hf",
)
model = AutoModelForCausalLM.from_pretrained(
    "tokyotech-llm/Swallow-7b-instruct-hf",
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
def get_answer(instruction):
  prompt = (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        f"### 指示:\n{instruction}\n\n### 応答:"
    )

  with torch.no_grad():
      token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
      output_ids = model.generate(
          token_ids.to(model.device),
          temperature=0.99,
          top_p=0.95,
          do_sample=True,
          max_new_tokens=1024,
      )
  output = tokenizer.decode(output_ids[0][token_ids.size(1) :])
  return output

In [4]:
%%time
instruction = "日本の首相は？"
print(get_answer(instruction))

私は何も知りません。</s>
CPU times: user 1.42 s, sys: 315 ms, total: 1.73 s
Wall time: 2.96 s


In [5]:
%%time
instruction = "アジアで有名な観光地と名物を詳しく述べてください。"
print(get_answer(instruction))

アジアにはいくつかの有名な観光地があります。インドのタージ・マハル、パキスタンのラワルピンディのバードシャー・モスク、ミャンマーのバガンの寺院、ベトナムのハノイのフエ・キャッスル、フィリピンのセブシティのサントニーニョ教会などが有名です。アジアでは、お茶やヌードル、特にインドやベトナムのカレーなどの伝統的な料理が有名です。</s>
CPU times: user 6.11 s, sys: 38.5 ms, total: 6.15 s
Wall time: 8.19 s


In [6]:
%%time
instruction = "日本で一番寒い地方はどこか教えてください。"
print(get_answer(instruction))

北海道</s>
CPU times: user 382 ms, sys: 36.4 ms, total: 418 ms
Wall time: 447 ms
